In [1]:
from  osgeo import ogr, osr
import psycopg2
import shapely
import shapely.wkt
import geopandas as gpd
%matplotlib inline

In [2]:
# connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
# cursor = connection.cursor()

# # cursor.execute("SELECT ST_ConvexHull(ST_Collect(d.the_geom))")
# cursor.execute('SELECT * FROM public.june1poly ORDER BY gid ASC LIMIT 100')
# result = cursor.fetchall()
# result

# try:
#     cursor.execute("ALTER TABLE public.june1poly ADD COLUMN centroid geometry(Point)")
# except psycopg2.ProgrammingError:
#     connection.rollback
# connection.commit()

# cursor.execute("UPDATE public.june1poly SET centroid=ST_Centroid(geom)")
# connection.commit()

# connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
# cursor = connection.cursor()
# try:
#     cursor.execute("ALTER TABLE public.june2poly ADD COLUMN centroid geometry(Point)")
# except psycopg2.ProgrammingError:
#     connection.rollback
# connection.commit()
# cursor.execute("UPDATE public.june2poly SET centroid=ST_Centroid(geom)")
# connection.commit()

In [3]:
def change_shrink(poly1, poly2, factor=0.5):
    newpoly = 'public.'+poly1[:-4]+poly2[:-4]+'shrink'
    
    connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
    cursor = connection.cursor()
    cursor.execute('DROP TABLE IF EXISTS '+newpoly)
    cursor.execute('CREATE TABLE '+newpoly+' (gid SERIAL PRIMARY KEY, avgscor numeric, numtwts integer, id integer, batchnm integer, geodata character varying(80) COLLATE pg_catalog."default", geom geometry(Polygon), centroid geometry(Point), changval numeric)')

    cursor.execute('INSERT INTO '+newpoly+' (avgscor, numtwts, geodata, id, batchnm, geom, centroid, changval)'+
                ' (SELECT DISTINCT j2.avgscor, j2.numtwts, j2.geodata, j2.id, j2.batchnm, j2.geom, j2.centroid,'+
                ' (ST_AREA(ST_INTERSECTION(j2.geom, j1.geom))/st_area(j2.geom))'+
                ' FROM public.'+poly1+' j1, public.'+poly2+' j2'+
                ' WHERE j1.id = j2.id AND ST_INTERSECTS(j1.geom, j2.geom) AND '+    
                    '(ST_AREA(ST_INTERSECTION(j2.geom, j1.geom))/st_area(j2.geom)) < '+str(factor)+') ')
    connection.commit()
    

In [4]:
def change_growth(poly1, poly2, factor=0.5):
    newpoly = 'public.'+poly1[:-4]+poly2[:-4]+'growth'
    
    connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
    cursor = connection.cursor()
    cursor.execute('DROP TABLE IF EXISTS '+newpoly)
    cursor.execute('CREATE TABLE '+newpoly+' (gid SERIAL PRIMARY KEY, avgscor numeric, numtwts integer, id integer, batchnm integer, geodata character varying(80) COLLATE pg_catalog."default", geom geometry(Polygon), centroid geometry(Point), changval numeric)')

    cursor.execute('INSERT INTO '+newpoly+' (avgscor, numtwts, geodata, id, batchnm, geom, centroid, changval)'+
                ' (SELECT DISTINCT j2.avgscor, j2.numtwts, j2.geodata, j2.id, j2.batchnm, j2.geom, j2.centroid,'+
                ' (ST_AREA(ST_INTERSECTION(j2.geom, j1.geom))/st_area(j2.geom))'+
                ' FROM public.'+poly1+' j1, public.'+poly2+' j2'+
                ' WHERE j1.id = j2.id AND ST_INTERSECTS(j1.geom, j2.geom) AND '+    
                    '(ST_AREA(ST_INTERSECTION(j2.geom, j1.geom))/st_area(j2.geom)) > '+str(factor)+') ')
    connection.commit()

In [5]:
def change_shift(poly1, poly2, distance=75000):
    newpoly = 'public.'+poly1[:-4]+poly2[:-4]+'shift'
    
    connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
    cursor = connection.cursor()
    cursor.execute('DROP TABLE IF EXISTS '+newpoly)
    cursor.execute('CREATE TABLE '+newpoly+' (gid SERIAL PRIMARY KEY, avgscor numeric, numtwts integer, id integer, batchnm integer, geodata character varying(80) COLLATE pg_catalog."default", geom geometry(Polygon), centroid geometry(Point), shiftdist numeric)')
    
    cursor.execute('INSERT INTO '+newpoly+' (avgscor, numtwts, geodata, id, batchnm, geom, centroid, shiftdist)'+
                '(SELECT j1.avgscor, j1.numtwts, j1.geodata, j1.id, j1.batchnm, j1.geom, j1.centroid, d.distval'+
                ' FROM public.'+poly1+' j1,'+
                    ' (SELECT ST_Distance_Spheroid(j1.centroid, j2.centroid,' +
                    ' \'SPHEROID["WGS 84",6378137,298.257223563]\') as distval, j1.id  AS jid'+
                    ' FROM public.'+poly1+' j1, public.'+poly2+' j2 '+
                    ' WHERE j1.id = j2.id AND '+
                    ' ST_Distance_Spheroid(j1.centroid, j2.centroid, \'SPHEROID["WGS 84",6378137,298.257223563]\') > '+str(distance)+') as d'+
               ' WHERE j1.id = d.jid)')
    connection.commit()

We create two generic queries to test what kind of results we are getting from the grow and shrink predicates.

First, growth:

In [36]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('SELECT DISTINCT j2.* '+
                'FROM public.june1poly j1, public.june2poly j2 '+
                'WHERE ST_INTERSECTS(j1.geom, j2.geom) AND '+    
                '(ST_AREA(ST_INTERSECTION(j2.geom, j1.geom))/st_area(j2.geom)) > .85')
growth = cursor.fetchall()
# growth

Next up, shrink:

In [37]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('SELECT DISTINCT j2.* '+
                'FROM public.june1poly j1, public.june2poly j2 '+
                'WHERE ST_INTERSECTS(j1.geom, j2.geom) AND '+    
                '(ST_AREA(ST_INTERSECTION(j2.geom, j1.geom))/st_area(j2.geom)) < .5')
shrink = cursor.fetchall()
# shrink

Now we create the Growth detector table:

In [25]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('DROP TABLE IF EXISTS public.june12growth')
cursor.execute('CREATE TABLE public.june12growth (gid SERIAL PRIMARY KEY, avgscor numeric, numtwts integer, id integer, batchnm integer, geodata character varying(80) COLLATE pg_catalog."default", geom geometry(Polygon), centroid geometry(Point), growval numeric)')
connection.commit()

We populate the growth detector table and include the % change.

The critical query here is:
>ST_INTERSECTS(j1.geom, j2.geom) AND ST_AREA(ST_INTERSECTION(j2.geom, j1.geom))/st_area(j2.geom)) > .85

In [26]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('INSERT INTO public.june12growth (avgscor, numtwts, geodata, id, batchnm, geom, centroid, growval)'+
                '(SELECT DISTINCT j2.avgscor, j2.numtwts, j2.geodata, j2.id, j2.batchnm, j2.geom, j2.centroid, '+
                ' (ST_AREA(ST_INTERSECTION(j2.geom, j1.geom))/st_area(j2.geom))'+
                ' FROM public.june1poly j1, public.june2poly j2'+
                ' WHERE j1.id = j2.id AND ST_INTERSECTS(j1.geom, j2.geom) AND '+    
                '(ST_AREA(ST_INTERSECTION(j2.geom, j1.geom))/st_area(j2.geom)) > .85)')
connection.commit()

Now we do the same but for shinking polygons:

In [21]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('DROP TABLE IF EXISTS public.june12shrink')
cursor.execute('CREATE TABLE public.june12shrink (gid SERIAL PRIMARY KEY, avgscor numeric, numtwts integer, id integer, batchnm integer, geodata character varying(80) COLLATE pg_catalog."default", geom geometry(Polygon), centroid geometry(Point), growval numeric)')
connection.commit()

We insert those polygons with an overlap area of less than 25%. This can be modified based on need.

In [22]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('INSERT INTO public.june12shrink (avgscor, numtwts, geodata, id, batchnm, geom, centroid, growval)'+
                ' (SELECT DISTINCT j2.avgscor, j2.numtwts, j2.geodata, j2.id, j2.batchnm, j2.geom, j2.centroid,'+
                ' (ST_AREA(ST_INTERSECTION(j2.geom, j1.geom))/st_area(j2.geom))'+
                ' FROM public.june1poly j1, public.june2poly j2'+
                ' WHERE j1.id = j2.id AND ST_INTERSECTS(j1.geom, j2.geom) AND '+    
                    '(ST_AREA(ST_INTERSECTION(j2.geom, j1.geom))/st_area(j2.geom)) < .6) ')
connection.commit()

Now we investigate if polygons are moving around.
This done by the comparing their centroids and checking if they have moved over 75km.

Remember these polygons are created through a convex hull of points, which cannot ensure the centroid will lie within the polygon itself. Which is why we are taking a sufficient;y large bounding value for the polygon.

In [56]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('SELECT ST_Distance_Spheroid(j1.centroid, j2.centroid,' +
                ' \'SPHEROID["WGS 84",6378137,298.257223563]\'), j1.id '+
                'FROM public.june1poly j1, public.june2poly j2 '+
                'WHERE j1.id = j2.id AND '+
                ' ST_Distance_Spheroid(j1.centroid, j2.centroid, \'SPHEROID["WGS 84",6378137,298.257223563]\') > 75000; ')
centroid_distance = cursor.fetchall()
connection.commit()
centroid_distance

[(89461.6239573092, 16), (112342.357817654, 2), (91443.0278208261, 14)]

We can see the shifing polygons are being detected. Now we create a table to store these.

In [57]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('DROP TABLE IF EXISTS public.june12shift')
cursor.execute('CREATE TABLE public.june12shift (gid SERIAL PRIMARY KEY, avgscor numeric, numtwts integer, id integer, batchnm integer, geodata character varying(80) COLLATE pg_catalog."default", geom geometry(Polygon), centroid geometry(Point), shiftdist numeric)')
connection.commit()

Now we carefully insert the polygons into the table:

In [63]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('INSERT INTO public.june12shift (avgscor, numtwts, geodata, id, batchnm, geom, centroid, shiftdist)'+
                '(SELECT j1.avgscor, j1.numtwts, j1.geodata, j1.id, j1.batchnm, j1.geom, j1.centroid, d.distval'+
                ' FROM public.june1poly j1,'+
                    ' (SELECT ST_Distance_Spheroid(j1.centroid, j2.centroid,' +
                    ' \'SPHEROID["WGS 84",6378137,298.257223563]\') as distval, j1.id  AS jid'+
                    ' FROM public.june1poly j1, public.june2poly j2 '+
                    ' WHERE j1.id = j2.id AND '+
                    ' ST_Distance_Spheroid(j1.centroid, j2.centroid, \'SPHEROID["WGS 84",6378137,298.257223563]\') > 75000) as d'+
               ' WHERE j1.id = d.jid)')
connection.commit()

In [6]:
#Comparing all sequential days
polygons =[]
for val in range(1,8):
    polygons.append('june'+str(val)+'poly')
for val in range(6):
    if val < 5:
        change_shrink(polygons[val],polygons[val+1])
        change_growth(polygons[val],polygons[val+1])
        change_shift(polygons[val],polygons[val+1])